# Data transforming and cleaning

In [1]:
import toLog
log = toLog.log('ETL process start')

In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName('ddam_project')
         .config('spark.some.config.option','some-value')
         .getOrCreate()
         )

In [3]:
# read file from hdfs and infer schema
df_raw = spark.read.csv("hdfs://masterbig-1.itc.unipi.it:54310/user/student18/Hotel_Reviews.csv", header = True, inferSchema = True)
df_raw.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



In [4]:
N_raw = df_raw.count()
print(N_raw)

515738


In [5]:
from pyspark.sql.functions import monotonically_increasing_id

In [6]:
# adding id
df_raw_id = df_raw.withColumn('id', monotonically_increasing_id())
df_raw_id.printSchema()
df_raw_id.take(1)

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- id: long (nullable = false)



[Row(Hotel_Address=' s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands', Additional_Number_of_Scoring=194, Review_Date='8/3/2017', Average_Score=7.7, Hotel_Name='Hotel Arena', Reviewer_Nationality=' Russia ', Negative_Review=' I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs t

## Reviews transformation

In [7]:
def catReviews(row):
    if row["Negative_Review"] == "No Negative" and row["Positive_Review"] == "No Positive":
        return "EMPTY"
    else:
        if row["Negative_Review"] == "No Negative":
            return (row["Positive_Review"].lower())
        elif row["Positive_Review"] == "No Positive":
            return (row["Negative_Review"].lower() )
        else:
            return(row["Negative_Review"].lower()  + ". " + row["Positive_Review"].lower())
            #users_ratings.append(row[12])       
        

In [8]:
def catReviewsLen(row, L = 11):
    out = ""
    if len(row["Negative_Review"]) > L:
        out += row["Negative_Review"]
    if len(row["Positive_Review"]) > L:
        out += '.' + row["Positive_Review"]
        
    return out.lower()
     

In [10]:
def correction(row):
    ''' Corrections and (some) stop words removal '''
    return (row.replace(" don t ", " don't ")
            .replace(" didn t ", " didn't ")
            .replace(" haven t ", " haven't ")
            .replace(" hadn t ", " hadn't ")
            .replace(" isn t ", " isn't ")
            .replace(" weren t ", " weren't ")
            .replace(" wasn t ", " wasn't ")
            .replace(" dont ", " don't ")
            .replace(" didnt ", " didn't ")
            .replace(" wont ", " won't ")
            .replace(" won t ", " won't ")
            .replace(" wouldn t ", " wouldn't ")
            .replace(" wouldnt ", " wouldn't ")
            .replace(" i ", " I ")
            .replace(" a "," ") # removing stop words 
            .replace(" self "," ")
            .replace(" it "," ")
            .replace(" bit "," ")
            .replace(" lot "," ")
            .replace(" also "," ")
            .replace(" wi-fi ","wifi")
            .replace(" s ","")
            .replace(" the "," ")
           )

In [ ]:
rdd_reviews = (df_raw_id.rdd
               #.sample(False, 0.10)
               .map(lambda x: (x['id'], catReviewsLen(x)))
               .filter(lambda x: len(x[1]) > 1) # remove empty reviews
               .map(lambda x: (x[0], correction(x[1]))) 
              )

In [ ]:
N_cat = rdd_reviews.count()
print(N_cat)

In [ ]:
# removed reviews:
N_raw - N_cat

In [ ]:
log.toLog( 'counting removed reviews: '+str(N_raw - N_cat))

In [ ]:
#rdd_reviews.filter(lambda x: len(x[1])<1 ).count()

# Keep only English reviews

In [ ]:
import langdetect as ld

In [ ]:
#detect english reviews


#reviews_rdd.map(ld.detect).take(10)
def detect_Eng(review):
    if (len(review) < 100 ):
        return True
    try:
        if ld.detect(review)== 'en':
            return True
        else:
            return False
    except:
        return True

In [ ]:
#stampa le reviews in lingua differente dall'inglese
#rdd_reviews.filter(lambda x: not detect_Eng(x[1])).take(30)

In [ ]:
#rdd_reviews.filter(lambda x: x[1] is None).count()

In [ ]:
log.toLog('starting detect_Eng')

In [ ]:
#remove EMPTY reviews, keep only english reviews
#creiamo un nuovo data frame(df_revs) con colonne id, Review

df_lang = (rdd_reviews.filter(lambda x: detect_Eng(x[1]))
           #.filter(lambda x: x[1] is not None)
           .toDF(['id','Review'])
          )
df_lang.printSchema()

In [ ]:
df_lang.take(3)

In [ ]:
N_cat_lang = df_lang.count()
N_cat_lang

In [ ]:
# Not english reviews removed:
N_cat - N_cat_lang

In [ ]:
#creiamo un nuovo dataframe con le review modificate, eliminando quelle "vecchie" e senza contare 
df_cleaned_lang = df_raw_id.join(df_lang, 'id', 'inner').drop("Positive_Review", "Negative_Review")
df_cleaned_lang.printSchema()

In [ ]:
N_cleaned_lang = df_cleaned_lang.count()
log.toLog('counted not english rev, removed:' + str(N_raw-N_cleaned_lang))
N_cleaned_lang

# Hotel nationality

In [12]:
import reverse_geocode
log.toLog('starting country extraction')

In [13]:
# example
coord =[ (43.6753176,10.5408628) ]
reverse_geocode.search(coord)

[{'city': 'Cascina', 'country': 'Italy', 'country_code': 'IT'}]

In [14]:
# check na value for lat/lng
#(df_cleaned_lang
# .sample(False, 0.01)
# .select('lat','lng').rdd.filter(lambda x: x['lng']== 'NA').count()
#)

In [15]:
df_coord = ( df_raw_id.select('id', 'lat', 'lng').rdd
            #.sample(False, 0.01)
            .filter(lambda x: x['lat']!='NA' and x['lng']!='NA')
            .map(lambda x: (x['id'], [ [float(x['lat']), float(x['lng'])] ] )) 
            .map(lambda x: (x[0], reverse_geocode.search(x[1])))
            .map(lambda x: (x[0] , x[1][0]['city'], x[1][0]['country'], x[1][0]['country_code']))
           ).toDF(['id','h_city','h_country','h_country_code'])


In [17]:
df_coord.show(10)

+----+-----------+--------------+--------------+
|  id|     h_city|     h_country|h_country_code|
+----+-----------+--------------+--------------+
| 183|  Amsterdam|   Netherlands|            NL|
| 405| Kensington|United Kingdom|            GB|
| 582| Kensington|United Kingdom|            GB|
| 822| Kensington|United Kingdom|            GB|
| 883| Kensington|United Kingdom|            GB|
| 887| Kensington|United Kingdom|            GB|
| 944| Kensington|United Kingdom|            GB|
| 946| Kensington|United Kingdom|            GB|
|1163|Clerkenwell|United Kingdom|            GB|
|1462|Clerkenwell|United Kingdom|            GB|
+----+-----------+--------------+--------------+
only showing top 10 rows



In [19]:
df_coord.createTempView('coord')

In [21]:
spark.sql("SELECT h_city, COUNT(h_city) AS n_city \
          FROM coord \
          GROUP BY h_city \
          ORDER BY n_city DESC \
          ").show()            
          

+-------------------+------+
|             h_city|n_city|
+-------------------+------+
|          Amsterdam|   481|
|             London|   383|
|         Kensington|   372|
|             Vienna|   357|
|          Bayswater|   340|
| West End of London|   338|
|              Milan|   324|
|              Paris|   310|
|     City of London|   211|
|             Poplar|   118|
|       Canary Wharf|   118|
|           Barbican|   110|
|   Levallois-Perret|   103|
|City of Westminster|   103|
|            Chelsea|    99|
|        Barri Gòtic|    77|
|            Wembley|    72|
|        Camden Town|    64|
|Dreta de l'Eixample|    59|
|      Bethnal Green|    47|
+-------------------+------+
only showing top 20 rows



In [22]:
hotel_countries = df_coord.select('h_country').distinct().rdd.map(lambda x: x['h_country']).collect()
hotel_countries

['France', 'Italy', 'Spain', 'Austria', 'United Kingdom', 'Netherlands']

In [25]:
#hotel_cities = df_coord.select('h_city').distinct().rdd.map(lambda x: x['h_city']).collect()
#hotel_cities

In [ ]:
# estrarre la nazionalità dall'indirizzo
def extractCountry(row):
    for country in hotel_countries:
        if country.lower() in row['Hotel_Address'].lower():
            return (row['id'], country )
    return 'EMPTY'
 

In [ ]:
rdd_hotel_countries = (df_raw_id
                       #.sample(False, 0.005)
                       .select('id', 'Hotel_Address').rdd.map(extractCountry)
                      )

In [ ]:
# check if all reviews have an associated hotel_country 
# rdd_hotel_countries.filter(lambda x: x[1] == 'EMPTY').count()

In [ ]:
# rdd_hotel_countries.take(1)

In [ ]:
df_hotel_countries = rdd_hotel_countries.toDF(['id', 'Hotel_Country'])

In [ ]:
print (df_hotel_countries.show())

## Join and save cleaned df to hdfs

In [ ]:
log.toLog( 'started final join and write to hdfs')

In [ ]:
df_cleaned = df_cleaned_lang.join(df_hotel_countries, 'id', 'inner')


In [ ]:
df_cleaned.write.csv("hdfs://masterbig-1.itc.unipi.it:54310/user/student18/df_cleaned.csv", header = True)

In [26]:
log.toLog( 'etl notebook finished')
log.close(' ')